In [21]:
import numpy as np
import pandas as pd
import gzip
import json
import matplotlib.pyplot as plt
import torch

import os
from os import walk
from transformers import BertTokenizer, AutoModel, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [2]:
#get path informationb
product_path = '../../../../src/data/product'
train_test_all_filtered_path = os.path.join(product_path, 'train_test_split/output_unfiltered_tables/large/after_manual_checking/baselines')


In [48]:
df_train = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_train'))
df_val = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_val'))
df_test = pd.read_csv(os.path.join(train_test_all_filtered_path,'df_test'))

In [49]:
df_train

,text,label
0,rum body lotion. a lightweight rum-scented bod...,736
1,Cisco Catalyst WS-C2960X-24TD-L Stackable Rack...,542
2,Laurel Heights Charcoal Crest LH99 Marchand's ...,1201
3,Laurel Heights Gray Summit LH98 Marchand's Int...,1312
4,Laurel Heights Gray Summit LH98 Marchand's Int...,1277
...,...,...
11116,Anthelios Xl Spf50 Gel-crema Toque Seco Color ...,905
11117,Vichy neovadiol crema piel seca 50ml Vichy neo...,48
11118,Anthelios Xl Leche Spf50+ 250ml ANTHELIOS XL S...,518
11119,Autohelios gel autobronceador 100ml AUTOHELIOS...,81


In [4]:
target_train = df_train['cluster_id_mapped']
target_val = df_val['cluster_id_mapped']
target_test = df_test['cluster_id_mapped']

In [5]:
x_train = df_train['concat'].to_list()
x_test = df_test['concat'].to_list()
x_val = df_val['concat'].to_list()
y_val = target_val.to_list()
y_train = target_train.to_list()
y_test = target_test.to_list()

In [29]:
x_val

['Silk Strands Shantung SS05 Kitchen Bath Floors & Design LLC in California has a top selection of American Olean Glass Tile Flooring, including Silk Strands™ Shantung in 12\\" x 12\\',
 'Laurel Heights Elevated Beige LH96 Kitchen Bath Floors & Design LLC in California has a top selection of American Olean Ceramic & Porcelain Tile Flooring, including Laurel Heights™ Elevated Beige in 12\\" x 18\\',
 'Crosswood Pelican CR96 Kitchen Bath Floors & Design LLC in California has a top selection of American Olean Glass Tile Flooring, including Crosswood™ Pelican in 12\\" x 12\\',
 'Laurel Heights Gray Summit LH98 Kitchen Bath Floors & Design LLC in California has a top selection of American Olean Ceramic & Porcelain Tile Flooring, including Laurel Heights™ Gray Summit in 2\\" x 4\\',
 'Jubilance Esprit JB03 Kitchen Bath Floors & Design LLC in California has a top selection of American Olean Glass Tile Flooring, including Jubilance™ Esprit in 2\\" x Random\\',
 'Silk Strands Georgette SS08 Kit

In [32]:
tokenizer = BertTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-squadv2")
model = AutoModel.from_pretrained("mrm8488/bert-tiny-finetuned-squadv2", num_labels=target_train.nunique())

https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /home/bizer-tp2021/.cache/huggingface/transformers/tmpzx0ed0ew


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/vocab.txt in cache at /home/bizer-tp2021/.cache/huggingface/transformers/453b49185fedfddb93809fc19cb1549f313cc2686b1a81fc2fd9298174836989.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /home/bizer-tp2021/.cache/huggingface/transformers/453b49185fedfddb93809fc19cb1549f313cc2686b1a81fc2fd9298174836989.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /home/bizer-tp2021/.cache/huggingface/transformers/tmp6b1rs7gq


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/special_tokens_map.json in cache at /home/bizer-tp2021/.cache/huggingface/transformers/92a2a4517aa2df06f5328fb21b6ecfb97ce7d85c5bd283593af7460e8063ffd4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /home/bizer-tp2021/.cache/huggingface/transformers/92a2a4517aa2df06f5328fb21b6ecfb97ce7d85c5bd283593af7460e8063ffd4.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /home/bizer-tp2021/.cache/huggingface/transformers/tmppvf49vqp


...

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/config.json in cache at /home/bizer-tp2021/.cache/huggingface/transformers/1c9c47debcf1ea704edc79a69cba3adee79cab3027129d23952abb913d834dc6.21458fa63aa73ccb6cf8f15144ae7279582dfa5bf1b3c4c7091f60b0a047af05
creating metadata file for /home/bizer-tp2021/.cache/huggingface/transformers/1c9c47debcf1ea704edc79a69cba3adee79cab3027129d23952abb913d834dc6.21458fa63aa73ccb6cf8f15144ae7279582dfa5bf1b3c4c7091f60b0a047af05
loading configuration file https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/config.json from cache at /home/bizer-tp2021/.cache/huggingface/transformers/1c9c47debcf1ea704edc79a69cba3adee79cab3027129d23952abb913d834dc6.21458fa63aa73ccb6cf8f15144ae7279582dfa5bf1b3c4c7091f60b0a047af05
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 

https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /home/bizer-tp2021/.cache/huggingface/transformers/tmpd80ecz69


Downloading:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

storing https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/pytorch_model.bin in cache at /home/bizer-tp2021/.cache/huggingface/transformers/f093b3e476e9145f97817223ffecf3e4385416b6f77d05dbd2181a70d17a01d7.4abee409e81e850e9dcabdff21722c70ffae48c5e4dfd6c1ea863e313becbf24
creating metadata file for /home/bizer-tp2021/.cache/huggingface/transformers/f093b3e476e9145f97817223ffecf3e4385416b6f77d05dbd2181a70d17a01d7.4abee409e81e850e9dcabdff21722c70ffae48c5e4dfd6c1ea863e313becbf24
loading weights file https://huggingface.co/mrm8488/bert-tiny-finetuned-squadv2/resolve/main/pytorch_model.bin from cache at /home/bizer-tp2021/.cache/huggingface/transformers/f093b3e476e9145f97817223ffecf3e4385416b6f77d05dbd2181a70d17a01d7.4abee409e81e850e9dcabdff21722c70ffae48c5e4dfd6c1ea863e313becbf24
Some weights of the model checkpoint at mrm8488/bert-tiny-finetuned-squadv2 were not used when initializing BertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initi

In [33]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = r'./TinyBert_Results',
    num_train_epochs=25,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,
    report_to='none',
    logging_dir=r'./TinyBert_logs',
    evaluation_strategy="epoch",
)

PyTorch: setting up devices


In [34]:
class dataDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [35]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [36]:
x_Train = tokenizer(x_train, truncation=True, padding='max_length')
x_Test = tokenizer(x_test, truncation=True, padding='max_length')
x_Val = tokenizer(x_val, truncation=True, padding='max_length')


train_data = dataDataset(x_Train, y_train)
test_data = dataDataset(x_Test, y_test)
val_data = dataDataset(x_Val, y_val)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [38]:
trainer.train()

***** Running training *****
  Num examples = 11121
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 34775


RuntimeError: stack expects each tensor to be equal size, but got [7] at entry 0 and [22] at entry 1

In [ ]:
trainer.evaluate(eval_dataset=test_data)